In [ ]:
%pip install matplotlib


In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from textblob import TextBlob
#import spacy 
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

# Add the scripts folder to the system path so we can import time_series.py
import sys
import os

# for a folder that contain scripts for modularity 
sys.path.append(os.path.abspath('../scripts'))
sys.path.append(os.path.abspath('../src'))
import path

from load_data import CSVData
from Data_visulization import EDA
from financial import FinancialAnalysis

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
folder_path = path.get_path_news()
file_load = CSVData(folder_path)
df = file_load.load_csv_file()
df.head()

In [ ]:
# removing Unnamed column from the dataset
df= df.loc[:,~df.columns.str.contains("Unnamed")]

In [4]:
df.info()

NameError: name 'df' is not defined

In [ ]:
# Calculate headline length
df['headline_length'] = df['headline'].str.len()
df['headline_length']

In [ ]:
# Descriptive statistics for headline length
headline_stats = df['headline_length'].describe().T
print("Descriptive Statistics for Headline Length:")
print(headline_stats)

In [ ]:

# Count articles by publisher
publisher_counts = df['publisher'].value_counts()

print("\nTop Publishers and their Article Counts:")
print(publisher_counts)

In [ ]:
# Analyze Publication Dates

# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')
# Drop rows with invalid or missing dates
df = df.dropna(subset=['date'])

# Group articles by publication date
daily_articles = df.groupby(df['date'].dt.date).size()

print(f"\nNumber of Articles Published by Date: {daily_articles}")

In [ ]:

# Plot articles by publisher
publisher_counts.head(20).plot(kind='bar', title='Top 20 Publishers by Article Count')
plt.xlabel('Publisher')
plt.ylabel('Number of Articles')
plt.show()

In [ ]:
# Drop rows where 'date' is NaT and explicitly create a copy
df1 = df.dropna(subset=['date']).copy()

# Extract additional date components for analysis
df1['year'] = df1['date'].dt.year
df1['month'] = df1['date'].dt.month
df1['day'] = df1['date'].dt.day

In [ ]:
# Count the number of articles per day
daily_counts = df1['date'].dt.date.value_counts().sort_index()

# Plot the trend of articles over time
plt.figure(figsize=(12, 6))
daily_counts.plot(kind='line', color='blue')
plt.title("Articles Published Over Time")
plt.xlabel("Date")
plt.ylabel("Number of Articles")
plt.xticks(rotation=45)
plt.show()

In [ ]:
 # Count articles by day of the 'month'
monthly_counts = df1['month'].value_counts().sort_index()

# Plot the frequency of articles by day of the 'month'
plt.figure(figsize=(12, 6))
monthly_counts.plot(kind='bar', color='orange', alpha=0.7)
plt.title("Articles Published by Day of the Month")
plt.xlabel("Day of the month")
plt.ylabel("Number of Articles")
plt.show()

In [ ]:
# Count articles by year
yearly_counts = df1['year'].value_counts().sort_index()

# Plot the frequency of articles by year
plt.figure(figsize=(10, 6))
yearly_counts.plot(kind='bar', color='green', alpha=0.7)
plt.title("Articles Published by year")
plt.xlabel("year")
plt.ylabel("Number of Articles")
plt.show()

In [ ]:
# Perform sentiment analysis
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'

df['sentiment'] = df['headline'].apply(get_sentiment)

# Count the number of articles for each sentiment
sentiment_counts = df['sentiment'].value_counts()
print(f"\nSentiment Analysis Results: {sentiment_counts}")

In [ ]:

# Bar chart for sentiment analysis
plt.figure(figsize=(8, 6))
sentiment_counts.plot(kind='bar', color=['green', 'blue', 'red'], alpha=0.7)

# Customize the plot
plt.title('Sentiment Analysis: Distribution of Sentiments', fontsize=16)
plt.xlabel('Sentiment', fontsize=12)
plt.ylabel('Number of Headlines', fontsize=12)
plt.xticks(rotation=0)  # Rotate x-axis labels if necessary
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:

# Define stopwords
stop_words = set(stopwords.words('english'))

# Tokenize and extract keywords
def extract_keywords(text):
    tokens = word_tokenize(text)
    keywords = [word.lower() for word in tokens if word.is_alpha() and word.lower() not in stop_words]
    return keywords

# Apply the function to the 'headline' column
#df['keywords'] = df['headline'].apply(extract_keywords)

In [ ]:
# Count most common keywords
all_keywords = [keyword for keywords in df['keywords'] for keyword in keywords]
common_keywords = Counter(all_keywords).most_common(20)

# Print the top 20 keywords
print(f"\nTop 20 Keywords: {common_keywords}")

In [ ]:

def analyze_publication_frequency(self, time_unit='D'):
        """ This function anlyzes how the publication frequency varies over time. The time can be, for daily,
        for weekly, for monthly, and for hourly frequency.
        """
        # Set 'date' as the index for time series analysis
        self.dataframe.set_index('date', inplace=True)

        # Resample by the specified time unit and count the number of articles published in each period
        publication_frequency = self.dataframe.resample(time_unit).size()

        # Plot the publication frequency over time
        plt.figure(figsize=(12, 6))
        publication_frequency.plot(kind='line', color='blue')
        plt.title(f"Publication Frequency Over Time ({time_unit})")
        plt.xlabel("Time")
        plt.ylabel("Number of Articles Published")
        plt.grid(True)
        plt.show()

In [ ]:
# 1. Analyze DAILY trends
# Extract the date (without time) and count articles published per day
df['publication_date'] = df['date'].dt.date
daily_trend = df.groupby('publication_date').size()

# Plot daily publication trends
plt.figure(figsize=(12, 6))
daily_trend.plot(kind='line', color='blue')
plt.title('Daily Publication Frequency')
plt.xlabel('Date')
plt.ylabel('Number of Articles Published')
plt.grid(True)
plt.show()

In [ ]:

# 2. Analyze MONTHLY trends
# Group data by year and month, and count articles
df['year_month'] = df['date'].dt.to_period('M')  # Format: YYYY-MM
monthly_trend = df.groupby('year_month').size()

# Plot monthly publication trends
plt.figure(figsize=(12, 6))
monthly_trend.plot(kind='line', color='green')
plt.title('Monthly Publication Frequency')
plt.xlabel('Year-Month')
plt.ylabel('Number of Articles Published')
plt.grid(True)
plt.show()

In [ ]:
# 3. Analyze YEARLY trends
# Group data by year and count articles
df['year'] = df['date'].dt.year
yearly_trend = df.groupby('year').size()

# Plot yearly publication trends
plt.figure(figsize=(12, 6))
yearly_trend.plot(kind='bar', color='orange')
plt.title('Yearly Publication Frequency')
plt.xlabel('Year')
plt.ylabel('Number of Articles Published')
plt.grid(axis='y')
plt.show()

In [ ]:

# Task 2: Identify unique domains if publishers contain email addresses
def extract_domain(publisher):
    if isinstance(publisher, str) and '@' in publisher:
        return publisher.split('@')[-1]
    return None

df['publisher_domain'] = df['publisher'].apply(extract_domain)
unique_domains = df['publisher_domain'].dropna().value_counts()

# Plot the top domains
plt.figure(figsize=(12, 6))
unique_domains.head(10).plot(kind='bar', color='teal')
plt.title('Top 10 Domains by Contribution')
plt.xlabel('Domain')
plt.ylabel('Number of Articles')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

In [ ]:
folder_path = path.get_path_price()

# Create an instance of CSVLoader
csv_loader = CSVData(folder_path)

# Load CSV files
csv_loader.load_data_files()

# Merge the loaded dataframes
merged_df = csv_loader.merge_dataframes()

# Display the first few rows
merged_df.head() 

In [ ]:
# Perform EDA for stock price
stock_price = EDA(merged_df)

In [ ]:
# Basic Information and Structure of stoke price
stock_price.display_info()

In [ ]:
stock_price.stat_summary()

In [ ]:
stock_price.univariate_cate()

In [ ]:
stock_price.univariate_num()

In [ ]:
# Correlation Analysis Using Heatmap of Correlation Matrix
stock_price.correlation_analysis()

In [ ]:
# Initialize the TimeSeries class with your dataframe
finan_data = FinancialAnalysis(merged_df)

In [ ]:
#SMA vs. Closing Price:
#Compare the SMA_20 with the Close price to identify trends.
finan_data.visualize_SMA()

In [ ]:
finan_data.visualize_RSI()

In [ ]:
finan_data.visualize_MACD()

In [ ]:
# Convert date columns to datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce') # news_data
merged_df['date'] = pd.to_datetime(merged_df['Date'], errors='coerce') # stock_data

In [ ]:
# Normalize timestamps to only keep the date (no time)
df['date'] = df['date'].dt.date   # news_data
merged_df['date'] = merged_df['date'].dt.date  # stock_data

In [ ]:
# Merge the two datasets on the date column
aligned_data = pd.merge(df, merged_df, on='date', how='inner')
aligned_data.head()